# Более подробно о возможностях Keras
В данной лекции мы с вами научимся пользоваться Keras'ом — частью уже озвученного фреймворка TensorFlow. Keras предназначен для упрощения разработки моделей и решения повседневных (для нейросетей) задач. В случае исследовательских работ или построения сложных схем лучше делать это на уровне TensorFlow — с использованием тензоров, самостоятельно определяя слои и способы их соединения. Keras же помогает быстро построить модель и обучить ее, используя одну из заранее составленных архитектур, или построив простую архитектуру самостоятельно.

Для начала проверим, что все необходимые библиотеки установлены. Кроме самого фреймворка, нам с вами еще понадобится пакет `tensorflow_datasets` – так мы быстро и удобно сможем получать доступ к популярным наборам данных для демонстрации возможностей обучения.

In [38]:
!pip install numpy tensorflow tensorflow_datasets

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


Далее импортируем нужные пакеты

In [39]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

Кроме того, давайте посмотрим, где и как в пакете keras расположены необходимые нам элементы.
К текущему моменту мы пока что рассмотрили только один тип слоев — полносвязный, однако в дальнейшей в рамках курса мы с вами изучим и другие. Все слои располагаются в модуле `layers`.

In [40]:
from tensorflow.keras import layers

Кроме того, мы с вами изучили некоторые функции активации, которые можно использовать при построении нейронных сетей. Все функции активации расположены в пакете `activations`.

In [41]:
from tensorflow.keras import activations

Также импортируем пакет `optimizers`, в котором расположены оптимизаторы для обучения.

In [42]:
from tensorflow.keras import optimizers

В модуле `losses` расположены различные функции потерь, которые можно использовать для обучения.

In [43]:
from tensorflow.keras import losses

И последнее, что нам понадобится импортировать — это метрики из пакета `metrics`. Функция потерь — это некоторая функция, которая возвращает абстрактный `loss`, по которому сложно и зачастую совершенно невозможно оценить качество модели. Для измерения метрик, понятных людям, воспользуемся модулем `metrics` и заданными там функциями.

In [44]:
from tensorflow.keras import metrics

Пакет `tensorflow_datasets` дает возможность скачивать многие открытые наборы данных. Информацию о данном пакете и список доступных наборов данных можно изучить здесь: https://www.tensorflow.org/datasets

Данный пакет нам пригодится для того, чтобы сделать ноутбук воспроизводимым на любом компьютере (конечно, с установленным софтом), и не зависеть от того, правильно ли вы скачаете данные для обучения. Сейчас мы подгрузим известный набор данных «Ирисы Фишера». Это простой набор данных, преимущественно используемый для обучения моделей классификации (в нашем случае -- классификации растений-ирисов). Каждый объект набора данных имеет 4 признака и относится к одному из трех классов.

Загрузим упомянутый набор данных, разделив на тренировочную и тестовую выборку по границе 80% (т.е. 80% данных будет использоваться для обучения и 20% для валидации модели). Кроме того, в данном случае необходимо указать флаг as_supervised для смены формата выдачи набора данных.

In [45]:
ds_train, ds_test = tfds.load(
    name='iris',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True
)

Давайте посмотрим на длины полученных тренировочного и валидационного набора данных и проверим, что все в порядке.

In [46]:
print(f"Lengths: {len(ds_train)}, {len(ds_test)}")

Lengths: 120, 30


Также давайте возьмем первые 5 записей из тренировочного набора данных и посмотрим на них.

In [47]:
examples = ds_train.as_numpy_iterator()
examples = [examples.next() for _ in range(5)]
features = [i[0] for i in examples]
classes = [i[1] for i in examples]

print("Features \t\t Classes")
for i in range(5):
    print(f"{features[i]} \t {classes[i]}")

Features 		 Classes
[5.1 3.4 1.5 0.2] 	 0
[7.7 3.  6.1 2.3] 	 2
[5.7 2.8 4.5 1.3] 	 1
[6.8 3.2 5.9 2.3] 	 2
[5.2 3.4 1.4 0.2] 	 0


2022-07-03 17:35:46.354241: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Как можно заметить, данные представлены в следующем формате: у каждого объекта набора данных имеются 4 признака и одно поле с кодом класса. Классов всего 3. Признаки представлены в удобном для нас формате (числа), и мы можем использовать их как есть (правда, неплохо бы их нормировать), а вот класс будет удобно преобразовать при помощи one-hot encoding для дальнейшего обучения.

Заведем константы для дальнейшего использования — размер входных данных, размер батча для обучения и количество классов для предсказания.

In [48]:
input_shape = (4, )     # Наши данные - одномерный массив с размерностью 4
batch_size = 10         # Размер батча на текущий момент можно выбрать любым, мы рассмотрим его оптимальный выбор далее в рамках курса
amount_of_classes = 3   # Количество классов определено данными — их 3i

Теперь произведем кодирование откликов: мы превратим один столбец с номером класса в 3 столбца, в одном из которых (соответствующем номеру класса) будет стоять единица, а в остальных — нули — это и есть процедура one-hot encoding'а.

Напишем отдельную функцию, которой мы и подействуем на наш набор данных. Она должна остаивть признаки объектов нетронутыми, а столбец, отвечающий классу, корректно преобразовать в озвученный ранее формат. Воспользуемся для этого встроенной функцией TensorFlow.

In [49]:
def make_one_hot(x, y):
    return x, tf.one_hot(y, depth=amount_of_classes)

Далее применим к тренировочному набору данных указанное преобразование, а также перемешаем его и нарежем на батчи указанного ранее нами размера. Стоит отметить флаг drop_remainder в конце — при определении батча, данный флаг поможет вам откинуть последний кусок данных, если он не совпадает по размеру с батчем, т.к. данных оказалось недостаточно (т.е. они не делятся нацело на размер батча).

In [50]:
ds_train = (
    ds_train
    .map(make_one_hot)
    .shuffle(len(ds_train))
    .batch(batch_size, drop_remainder=False)
)
    
ds_test = (
    ds_test
    .map(make_one_hot)
    .batch(batch_size, drop_remainder=False)
)

Теперь пора создать нашу модель. Сейчас мы будем использовать модель типа Sequential — т.е. модель, в которой слои идут последовательно, и каждый слой связан только с предыдущим и последующим слоями. Существуют и более сложные архитектуры, мы с вами их рассмотрим позднее.

Итак, создадим нашу модель и начнем постепенно ее наполнять.

In [51]:
model = keras.Sequential()

Далее с помощью функции `.add` можно присоединять к ней объекты-слои. Фреймворк будет автоматически связывать их с последующими и предыдущими. Первый слой, который мы с вами добавим, это специальный объект типа `InputLayer` — слой, определяющий размер входных данных и размер пакета передаваемых данных.

In [52]:
model.add(layers.InputLayer(input_shape = input_shape, batch_size = batch_size))

После этого можно добавить несколько полносвязных слоев. В каждом слое необходимо указать количество нейронов, а также функцию активации. Функцию активации можно импортировать из пакета `activations`, о котором говорилось ранее. Также функцию активации можно указать текстовой строкой с соответсвующим названием (sigmoid, relu, tanh, и т.д.)

In [53]:
model.add(layers.Dense(32, activation = activations.sigmoid))
model.add(layers.Dense(16, activation = 'sigmoid'))

Последним слоем добавим слой с количеством нейронов, равным количеству классов. В качестве функции активации укажем уже много раз встречавшуюся функцию softmax.

In [54]:
model.add(layers.Dense(amount_of_classes, activation = activations.softmax))

После определения структуры модели ее необходимо скомпилировать. Используя возможности фреймворка TensorFlow, этот процесс создает по заданной модели соответствующий граф вычислений.

В качестве параметров компиляции необходимо передать несколько параметров. Во-первых, экземпляр оптимизатора — создадим для этого новый оптимизатор Adam с коэффициентом скорости обучения $0.003$. Кроме того, необходимо задать функцию потерь. Так как в нашем случае мы решаем задачу многоклассовой классификации, соответствующая функция потерь называется CategoricalCrossentropy. В качестве еще одного аргумента передадим список метрик, которые нам хотелось бы вычислять в процессе обучения — в данном случае укажем только точность.

In [55]:
model.compile(
    optimizer = optimizers.Adam(learning_rate = 0.003),
    loss = losses.CategoricalCrossentropy(),
    metrics = [metrics.CategoricalAccuracy()]
)

Процесс обучения модели запускается с помощью функции `fit`. В данную функцию мы подадим сам набор данных для обучения, количество эпох и набор данных для валидации. Функция запустит и обучит модель, а также вернет результаты в виде объекта `keras.History`.

In [56]:
history = model.fit(ds_train, epochs=20, validation_data=ds_test)

Epoch 1/20


2022-07-03 17:35:46.600162: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - 2s 20ms/step - loss: 1.4831 - categorical_accuracy: 0.3333 - val_loss: 1.2185 - val_categorical_accuracy: 0.3333
Epoch 2/20
12/12 [==============================] - 0s 8ms/step - loss: 1.1872 - categorical_accuracy: 0.2250 - val_loss: 1.0965 - val_categorical_accuracy: 0.2667
Epoch 3/20
 1/12 [=>............................] - ETA: 0s - loss: 1.0602 - categorical_accuracy: 0.2000

2022-07-03 17:35:48.693330: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - 0s 9ms/step - loss: 1.0882 - categorical_accuracy: 0.3500 - val_loss: 1.0791 - val_categorical_accuracy: 0.6333
Epoch 4/20
12/12 [==============================] - 0s 9ms/step - loss: 1.0631 - categorical_accuracy: 0.6667 - val_loss: 1.0509 - val_categorical_accuracy: 0.6333
Epoch 5/20
12/12 [==============================] - 0s 8ms/step - loss: 1.0268 - categorical_accuracy: 0.6750 - val_loss: 1.0053 - val_categorical_accuracy: 0.6333
Epoch 6/20
12/12 [==============================] - 0s 8ms/step - loss: 0.9827 - categorical_accuracy: 0.6750 - val_loss: 0.9602 - val_categorical_accuracy: 0.6333
Epoch 7/20
12/12 [==============================] - 0s 9ms/step - loss: 0.9355 - categorical_accuracy: 0.6750 - val_loss: 0.9148 - val_categorical_accuracy: 0.6333
Epoch 8/20
12/12 [==============================] - 0s 9ms/step - loss: 0.8817 - categorical_accuracy: 0.6750 - val_loss: 0.8629 - val_categorical_accuracy: 0.6333
Epoch 9/20
12/12 [=========

Данный объект (`keras.History`) содержит в себе историю процесса обучения, использованные параметры и ссылки на модель и объекты, использованные в процессе обучения. К примеру, можно вывести параметры обучения:

In [57]:
print(f"Params of training: {history.params}")

Params of training: {'verbose': 1, 'epochs': 20, 'steps': 12}


Также можно узнать, какой коэффициент скорости обучения использовался:

In [58]:
print(f"Used learning rate: {history.model.optimizer.lr}")

Used learning rate: <tf.Variable 'Adam/learning_rate:0' shape=() dtype=float32, numpy=0.003>


Или даже посмотреть на веса обученного первого слоя модели:

In [59]:
print(f"Weights of first layer: {history.model.layers[0].weights}")

Weights of first layer: [<tf.Variable 'dense_6/kernel:0' shape=(4, 32) dtype=float32, numpy=
array([[-0.26571333,  0.09799431,  0.1230574 , -0.40980718,  0.13215396,
         0.00855689, -0.021257  ,  0.50601727,  0.26469764, -0.00977292,
        -0.24450903, -0.21784444, -0.30268455,  0.03998473,  0.13658999,
         0.01240103,  0.2206359 , -0.19532038, -0.09640728,  0.21457122,
         0.11764064, -0.22656752, -0.31943744,  0.6015011 ,  0.12573062,
        -0.09823257,  0.13728398, -0.04390372, -0.01091221,  0.35040778,
         0.20212397,  0.0201603 ],
       [-0.24112242, -0.5713144 ,  0.35579786,  0.52292866,  0.33045575,
         0.46320075,  0.56905216, -0.28622776,  0.47271198,  0.12396912,
         0.0944686 , -0.5852205 ,  0.45902562, -0.02700393,  0.4704883 ,
         0.57751787,  0.49774513,  0.5717942 ,  0.06454657,  0.18951973,
         0.5149319 ,  0.08325543,  0.03675253, -0.04613746,  0.4806617 ,
        -0.38969198, -0.10281853, -0.30580047,  0.4222176 ,  0.164126

Далее обученную модель можно сериализовать с помощью средств фреймворка или использовать для предсказания на существующем сформированном наборе данных. Результатом предсказания будут значения трех нейронов каждого из классов. Значения — это соответствующие вероятностям принадлежности экземпляра соответствующему классу.

In [60]:
model.predict(ds_test)


3/3 [==============================] - 0s 2ms/step


2022-07-03 17:35:50.821399: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


array([[0.88755757, 0.08527765, 0.0271648 ],
       [0.88241106, 0.08924688, 0.02834208],
       [0.00709296, 0.31292704, 0.67998004],
       [0.8883408 , 0.08474535, 0.02691392],
       [0.01906477, 0.42576474, 0.5551705 ],
       [0.01093391, 0.3607154 , 0.6283507 ],
       [0.86752766, 0.10057039, 0.03190193],
       [0.01399535, 0.3889148 , 0.5970899 ],
       [0.01094799, 0.36363226, 0.6254198 ],
       [0.01517392, 0.3997789 , 0.5850472 ],
       [0.02753344, 0.4644989 , 0.5079677 ],
       [0.05796535, 0.52952904, 0.41250563],
       [0.02613451, 0.45905235, 0.5148132 ],
       [0.881033  , 0.090222  , 0.02874492],
       [0.15035309, 0.5534708 , 0.29617614],
       [0.06458768, 0.54077387, 0.39463854],
       [0.03775039, 0.49780568, 0.46444395],
       [0.02246172, 0.4428921 , 0.5346463 ],
       [0.8825881 , 0.08914482, 0.02826711],
       [0.01902623, 0.4247732 , 0.5562006 ],
       [0.05508385, 0.52810454, 0.41681165],
       [0.01385774, 0.39171046, 0.59443176],
       [0.